# Введение
На основе базы данных - Startup Investments, опубликованной на популярной платформе для соревнований по исследованию данных Kaggle и храниящей информацию о венчурных фондах и инвестициях в компании-стартапы - произвести различные загрузки данных венчурных фондов с использованием SQL
___
**introduction**

Based on data - Startup Investments, published on the data mining platform Kaggle and storing information about venture capital funds and investments in start-up companies - quantitative data on venture investments using SQL

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
import warnings; warnings.filterwarnings(action='once')


**1 Cчитаем сколько компаний закрылось**
___
1 Count how many companies closed

In [ ]:
#считаем сколько компаний закрылось
# counting how many companies closed
SELECT COUNT(status)
FROM company
WHERE status = 'closed';

**2 Отобразим количество привлечённых средств для новостных компаний США из таблицы company.** 
Отсортируем таблицу по убыванию значений в поле funding_total.
___
Let's display the amount of funds raised for US news companies from the company table. Sort the table in descending order of values in the funding_total field.

In [ ]:
SELECT funding_total
FROM company
WHERE category_code = 'news' AND country_code = 'USA'
ORDER BY funding_total DESC ;

**3 Найдем общую сумму сделок по покупке одних компаний другими в долларах.** 

Отберем сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.
___
Find the total amount of transactions for the purchase of some companies by others in dollars.

We will select transactions that were carried out only for cash from 2011 to 2013 inclusive.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash' AND 
EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN '2011' AND '2013' ;

**4 Отобразим имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.**
___
Let's display the first name, last name, and account names of people on Twitter whose account names begin with 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%' ;

**5 Выведим на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.**
___
Let's display all the information about people whose twitter account names contain the substring 'money' and whose last name starts with 'K'.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%' AND last_name LIKE 'K%' ;

**6 Для каждой страны отобразим общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране.**

Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируем данные по убыванию суммы.
___
For each country, we will display the total amount of attracted investments received by companies registered in this country.

The country in which the company is registered can be identified by the country code. Sort the data in descending order.

In [ ]:
SELECT SUM(funding_total) AS total_sum,
       country_code
FROM company
GROUP BY country_code
ORDER BY total_sum DESC ;

**7 Составим таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.**

Оставим в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.
___
Let's make a table that will include the date of the round, as well as the minimum and maximum values of the amount of investments attracted on this date.

Let's leave in the final table only those records in which the minimum value of the investment amount is not equal to zero and is not equal to the maximum value.

In [ ]:
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != 0 AND MIN(raised_amount) <> MAX(raised_amount)  ;

**8 Создадим поле с категориями:**

Для фондов, которые инвестируют в 100 и более компаний - high_activity.

Для фондов, которые инвестируют в 20 и более компаний до 100 - middle_activity.

Если количество инвестируемых компаний фонда не достигает 20 - low_activity.
___

Let's create a field with categories:

For funds that invest in 100 or more companies - high_activity.

For funds that invest in 20 or more companies up to 100 - middle_activity.

If the number of invested fund companies does not reach 20 - low_activity.

In [ ]:
SELECT *,
       CASE
           WHEN invested_companies >= 100 THEN 'high_activity' 
           WHEN invested_companies < 20 THEN 'low_activity'
           ELSE 'middle_activity'
       END
FROM fund ;

**9 Для каждой из категорий, посчитаем округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие**
___
For each of the categories, we will calculate the average number of investment rounds, rounded to the nearest whole number, in which the fund has participated

In [ ]:
SELECT CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds)) AS avg_investment_rounds
FROM fund
GROUP BY activity
ORDER BY avg_investment_rounds
;

**10 Проанализируем, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.** 
___
Let's analyze in which countries there are funds that most often invest in startups

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies) AS avg_ic
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) != 0
ORDER BY avg_ic DESC, country_code
LIMIT 10

**11 Отобразим имя и фамилию всех сотрудников стартапов и поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.**
___
Display the first and last names of all startup employees and the field with the name of the educational institution that the employee graduated from, if this information is known

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON e.person_id=p.id  ;

**12 Для каждой компании найдем количество учебных заведений, которые окончили её сотрудники**
___
For each company, find the number of educational institutions that its employees graduated from**

In [ ]:
SELECT c.name,
       COUNT(DISTINCT e.instituition) as count_instituition
FROM company AS c
JOIN people AS p ON c.id=p.company_id
JOIN education AS e ON e.person_id=p.id
GROUP BY c.name 
ORDER BY count_instituition DESC 
LIMIT 5;

**13 Составим список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.**
___
Let's make a list with unique names of closed companies for which the first funding round turned out to be the last

In [ ]:
SELECT DISTINCT name
FROM company AS c

INNER JOIN funding_round AS rf ON rf.company_id=c.id
WHERE c.status = 'closed' 
AND is_first_round = 1 
AND is_last_round = 1

GROUP BY name;

**14 Составим список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.**
___
Let's make a list of unique numbers of employees who work in the companies selected in the previous task

In [ ]:
SELECT p.id
FROM people AS p
WHERE p.company_id IN (SELECT c.id
FROM company AS c
INNER JOIN funding_round AS rf ON rf.company_id=c.id
WHERE c.status = 'closed' 
AND is_first_round = 1 
AND is_last_round = 1
GROUP BY c.id) ;

**15 Составим таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.**
___
Let's make a table that will include unique pairs with employee numbers from the previous task and the educational institution that the employee graduated from

In [ ]:
SELECT DISTINCT p.id,
       e.instituition
       
FROM people AS p
INNER JOIN company AS c ON p.company_id = c.id
INNER JOIN education AS e ON p.id = e.person_id
WHERE c.name IN (SELECT c.name
FROM company AS c
INNER JOIN funding_round AS rf ON rf.company_id=c.id
WHERE c.status = 'closed' 
AND is_first_round = 1 
AND is_last_round = 1
GROUP BY c.name) ;

**16 Посчитаем количество учебных заведений для каждого сотрудника из предыдущего задания. Учтем, что некоторые сотрудники могли окончить одно и то же заведение дважды.**
___
Let's count the number of educational institutions for each employee from the previous task. We take into account that some employees could graduate from the same institution twice

In [ ]:
SELECT p.id,
       COUNT(e.instituition)
       
FROM people AS p
JOIN company AS c ON p.company_id = c.id
JOIN education AS e ON p.id = e.person_id
WHERE p.company_id IN (SELECT c.id
FROM company AS c
INNER JOIN funding_round AS fr ON fr.company_id=c.id
WHERE c.status = 'closed' 
AND is_first_round = 1 
AND is_last_round = 1
GROUP BY c.id) 
GROUP BY p.id ;

**17 Дополним предыдущий запрос - выведем среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний.**
___
Let's supplement the previous query - we will display the average number of educational institutions (all, not only unique ones) that employees of different companies graduated from.**

In [ ]:
WITH
count_i AS (SELECT p.id,
       COUNT(e.instituition) AS count_inst
FROM people AS p
JOIN company AS c ON p.company_id = c.id
JOIN education AS e ON p.id = e.person_id
WHERE p.company_id IN (SELECT c.id
FROM company AS c
INNER JOIN funding_round AS fr ON fr.company_id=c.id
WHERE c.status = 'closed' 
AND is_first_round = 1 
AND is_last_round = 1
GROUP BY c.id) 
GROUP BY p.id)
                      
SELECT AVG(count_inst)
FROM count_i ;

**18 Выведем среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook.**
___
Let's display the average number of educational institutions (all, not just unique ones) that Facebook employees graduated from

In [ ]:
WITH
count_i AS (SELECT p.id,
       COUNT(e.instituition) AS count_inst
FROM people AS p
JOIN company AS c ON p.company_id = c.id
JOIN education AS e ON p.id = e.person_id
WHERE p.company_id IN (SELECT c.id
FROM company AS c
INNER JOIN funding_round AS fr ON fr.company_id=c.id
WHERE c.name LIKE '%Facebook%' 
GROUP BY c.id) 
GROUP BY p.id)
                      
SELECT AVG(count_inst)
FROM count_i ;

**19 Составим таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.** 

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.
___
Let's create a table from the fields:
name_of_fund - name of the fund;
name_of_company - company name;
amount — the amount of investments that the company raised in the round.**

The table will include data on companies whose history had more than six important milestones, and funding rounds took place from 2012 to 2013 inclusive

In [ ]:
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount 
FROM investment AS i
         JOIN company AS c ON i.company_id=c.id
         JOIN fund AS f ON f.id=i.fund_id 
         JOIN funding_round AS fr ON fr.id = i.funding_round_id
         WHERE c.milestones > 6
         AND 
         EXTRACT(YEAR FROM CAST(funded_at AS date)) 
         BETWEEN 2012 AND 2013 ;

**20 Выгрузим таблицу, в которой будут такие поля:**
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не будем учитывать те сделки, в которых сумма покупки равна нулю.
___
Let's upload a table with the following fields:
- name of the company-buyer;
- transaction amount;
- the name of the company that was bought;
- the amount of investments invested in the acquired company;
- share, which shows how many times the purchase amount exceeded the amount invested in the company, rounded to the nearest whole number.

We will not take into account those transactions in which the purchase amount is equal to zero.

In [ ]:
SELECT c1.name,
       a.price_amount,
       c2.name,
       c2.funding_total,
       ROUND(a.price_amount / c2.funding_total)
FROM acquisition AS a
JOIN company AS c1 ON a.acquiring_company_id=c1.id 
JOIN company AS c2 ON a.acquired_company_id=c2.id
WHERE a.price_amount > 0 
AND c2.funding_total > 0
ORDER BY a.price_amount DESC, c2.name
LIMIT 10 ;

**21 Выгрузим таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно.** 

Выведем также номер месяца, в котором проходил раунд финансирования.
___
Download a table that will include the names of companies from the social category that received funding from 2010 to 2013 inclusive.

We will also display the number of the month in which the funding round took place.

In [ ]:
SELECT c.name,
       EXTRACT(MONTH FROM fr.funded_at) AS month
FROM company AS c
JOIN funding_round AS fr ON fr.company_id=c.id
WHERE c.category_code = 'social' 
AND EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013
AND fr.raised_amount != 0 ;

**22 Отберем данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды.**

Предусмотрим поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.
___
Let's select data by months from 2010 to 2013, when investment rounds were held.

Consider the fields:
- the number of the month in which the rounds took place;
- the number of unique fund names from the US that have invested this month;
- the number of companies purchased this month;
- the total amount of purchase transactions in this month.

In [ ]:
WITH
t_1 AS (SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS number_of_month,
               COUNT(DISTINCT(f.name)) AS count_unique_fund
        FROM funding_round AS fr
        JOIN investment AS i ON fr.id = i.funding_round_id
        JOIN fund AS f ON i.fund_id = f.id
        WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2010 AND 2013
        AND f.country_code LIKE '%USA%'
        GROUP BY number_of_month),
        
t_2 AS (SELECT COUNT(a.id) AS count_buying_company,
               SUM(a.price_amount) AS sum_byuing,
               EXTRACT(MONTH FROM CAST(a.acquired_at AS date)) AS number_of_month
        FROM acquisition AS a
        WHERE EXTRACT(YEAR FROM CAST(a.acquired_at AS date)) BETWEEN 2010 AND 2013
        GROUP BY number_of_month)
SELECT t_1.number_of_month,
       t_1.count_unique_fund,
       t_2.count_buying_company,
       t_2.sum_byuing
FROM t_1 JOIN t_2 ON t_1.number_of_month = t_2.number_of_month ;


**23 Составим сводную таблицу и выведем среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах.**
___
Let's compile a pivot table and display the average amount of investments for countries that have startups registered in 2011, 2012 and 2013.

In [ ]:
WITH
a_2011 AS (SELECT country_code AS country,
                  AVG(funding_total) AS total_avg_2011  
          FROM company 
          WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
          GROUP BY country),
                  
b_2012 AS (SELECT country_code AS country,
                  AVG(funding_total) AS total_avg_2012
          FROM company 
          WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
          GROUP BY country),        
           
c_2013 AS (SELECT country_code AS country,
                  AVG(funding_total) AS total_avg_2013
          FROM company 
          WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
          GROUP BY country)
   
SELECT a_2011.country,  
       total_avg_2011,
       total_avg_2012,
       total_avg_2013
FROM a_2011
INNER JOIN b_2012 ON a_2011.country = b_2012.country
INNER JOIN c_2013 ON b_2012.country = c_2013.country
ORDER BY total_avg_2011 DESC; 